Import packages needed

In [2]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
#sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')
sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\CodeAndPackages\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [3]:
#importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
#ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsettingG6.yml','ops3Planes.npy')
SaveFolder=FBS.suite2pInitiate(ops0,ConfigFolder)
binFile = glob.glob(ops0['save_path0'] + '/TSeries*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
#Load Data
print(binFile)
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)
rawBin.shape

Processed data would be saved inE:\LuSLMOnlineTest\SL0855-Emx1G6CII-AAV9CAMKII\02262025\suite2p
File found: C:\Users\zhangl33\Projects\Project-SLMonlineControl\config\SLMsetting.yml
Copying to: E:\LuSLMOnlineTest\SL0855-Emx1G6CII-AAV9CAMKII\02262025\CurrentSLMsetting.yml
Current SLMsetting copied successfully!
['E:\\LuSLMOnlineTest\\SL0855-Emx1G6CII-AAV9CAMKII\\02262025\\TSeries-02262025-0930-001.bin']


(12300, 512, 512)

In [18]:
ops0['spatial_scale']=2
ops0['threshold_scaling']=1.4
ops0['denoise']=1

In [10]:
confSet

{'save_path0': 'E:\\LuSLMOnlineTest\\SL0855-Emx1G6CII-AAV9CAMKII\\02262025\\',
 'SLM_Pixels_X': 512,
 'SLM_Pixels_Y': 512,
 'scan_Z': 196.6,
 'ETL': '-50 0 50',
 'IsSpiral': 'True',
 'SpiralSizeUM': 15,
 'SpiralRevolution': 3,
 'Repetition': 5,
 'Duration': 20,
 'InterPointDelay': 30,
 'UncagingLaserPower': '1.4 1.5 1.6',
 'InitialDelay': 22.1,
 'NumNonTarget': 20,
 'NumNTperTrial': 9,
 'NumTrial': 5,
 'RadiusAvoidParam': 3,
 'nchannels': 1,
 'fs': 6.9,
 'tau': 1.5,
 'Ziteration': 11,
 'ZRepetition': 34}

Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [19]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)
FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Binning movie in chunks of length 10
Binned movie of size [410,512,512] created in 2.23 sec.
Binned movie denoised (for cell detection only) in 9.41 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 14.00 
0 ROIs, score=158.31
Detected 68 ROIs, 4.43 sec
After removing overlaps, 68 ROIs remain
added enhanced mean image
Masks created, 0.21 sec.
Extracted fluorescence from 68 ROIs in 4100 frames, 2.74 sec.
Extracted fluorescence from 68 ROIs in 4100 frames, 2.84 sec.
['skew', 'npix_norm', 'compact']
Processing plane 1
Binning movie in chunks of length 10
Binned movie of size [410,512,512] created in 2.24 sec.
Binned movie denoised (for cell detection only) in 9.55 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 14.00 
0 ROIs, score=140.63
Detected 108 ROIs, 5.11 sec
After removing overlaps, 106 ROIs remain
added enhanced mean image
Masks created, 0.29 sec.
Extracted fluorescence from 106 ROIs in 4100 frames, 3.24

Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [21]:
FBS.PostMannual(SaveFolder,ops0)

284 of units including 207 identified cells in combined planes
